In [1]:
from becquerel import Spectrum
import numpy as np
import matplotlib.pyplot as plt
import becquerel as bq
import scipy.integrate as integrate
import math as m
import importlib
import os

In [2]:
import analysis_methods as am
import PF
import ROI
importlib.reload(am)
importlib.reload(PF)
importlib.reload(ROI)

C:\Users\Daniel\Anaconda3\lib\site-packages\html5lib\_trie\_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


<module 'ROI' from 'C:\\Users\\Daniel\\Desktop\\Radwatch\\radwatch-analysis\\ROI.py'>

In [3]:
def plot_spec(spectrum, xmode='channel'):
    if xmode == 'channel':
        facecolor = 'green'
    else:
        facecolor = 'blue'
    plt.figure()
    spectrum.fill_between(xmode=xmode, facecolor=facecolor, alpha=0.4, ax=plt.gca())
    spectrum.plot('k-', lw=0.7, xmode=xmode, ax=plt.gca())
    if xmode == 'channel':
        plt.xlim(0, spectrum.bin_edges_raw.max())
        plt.title('Uncalibrated spectrum')
    else:
        plt.xlim(0, spectrum.energies_kev[-1])
        plt.title('Calibrated spectrum')
    plt.yscale('log')
    plt.ylim(2e-1)
    plt.tight_layout()
def plot_calibrator(cal):
    cal.peakfinder.spectrum.apply_calibration(cal.cal)
    print('fit gain:', cal.gain, 'keV/channel')
    print('fit channels:', cal.fit_channels)

    plt.figure()
    plt.title('Peaks used in fit')
    cal.plot()
    plt.tight_layout()
    #plt.savefig('peaks used in fit.png',dpi = 200)

    plot_spec(cal.peakfinder.spectrum, xmode='channel')
    for x, erg in zip(cal.fit_channels, cal.fit_energies):
        chan = cal.peakfinder.spectrum.find_bin_index(x, use_kev=False)
        y = cal.peakfinder.spectrum.counts_vals[chan-10:chan+10].max() * 1.5
        plt.plot([x, x], [1e-1, y], 'r-', alpha=0.5)
        plt.text(x, y, '{:.1f} keV'.format(erg))
    #plt.savefig('Uncal spec.png',dpi = 200)


    plot_spec(cal.peakfinder.spectrum, xmode='energy')
    for erg in cal.fit_energies:
        x = int(erg / cal.gain)
        chan = cal.peakfinder.spectrum.find_bin_index(x, use_kev=False)
        y = cal.peakfinder.spectrum.counts_vals[chan-15:chan+15].max() * 1.5
        plt.plot([erg, erg], [1e-1, y], 'r-', alpha=0.5)
        plt.text(erg, y, '{:.1f} keV'.format(erg))
    #plt.savefig('cal spec.png',dpi=200)
        
        


In [4]:
file_path = r'C:\Users\ahanks\Google Drive\RadWatch\Gamma\Gamma Spectra\Fish Samples 2019'
spec = file_path + r'\efficiency_calibration_2020-3-5.Spe'
bg = file_path + r'\Background_Spectrum_12_20_19.Spe'
spec = bq.Spectrum.from_file(spec)
bg = bq.Spectrum.from_file(bg)

AssertionError: 

In [ ]:
# apply the kernel to the data to get SNR
kernel = bq.GaussianPeakFilter(3700, 10, 5)
finder = bq.PeakFinder(spec, kernel)
cal = bq.AutoCalibrator(finder)

In [ ]:
cal.peakfinder.find_peaks(min_snr=15, xmin=400)
print(cal.peakfinder.channels)
plt.figure()
plt.title('find_peaks')
cal.peakfinder.plot()
plt.tight_layout()
#plt.savefig('find_peaks_spec',dpi = 200)

In [ ]:
# perform calibration
cal.fit(
    [1435.795, 1460.82,2614.51],
    optional=[201.83,306.78, 351.93, 338.32,569.33,604.721,788.742,795.864, 911.20, 1120.294, 1620.50, 1764.49, 2118.514],
    gain_range=[0.3, 10],
    de_max=100.,
)
plot_calibrator(cal)
spectrum = cal.peakfinder.spectrum


In [ ]:
kernel = bq.GaussianPeakFilter(3700, 10, 5)
finder = bq.PeakFinder(bg, kernel)
cal = bq.AutoCalibrator(finder)
cal.peakfinder.find_peaks(min_snr=15, xmin=200)
print(cal.peakfinder.channels)
plt.figure()
plt.title('find_peaks')
cal.peakfinder.plot()
plt.tight_layout()
#plt.savefig('find peaks bg.png',dpi=200)

In [ ]:
# perform calibration
cal.fit(
    [1460.82, 1764.49],
    optional=[186.21,238.63,609.32,911.20,242.00,295.22,338.32,351.93,1620.50],
    gain_range=[2.5e-2, 10],
    de_max=100.,
)
plot_calibrator(cal)
background = cal.peakfinder.spectrum

In [ ]:
#Find Lu-176, La-138, K-40
source_isotopes = np.array(['Lu_176_1','Lu_176_2','Lu_176_3',
                            'La_138_1','La_138_2',
                           'K-40']); #get counts under the peak subtrack the background
source_energies = np.array([75,186,294,
                           782,1435.795,
                           1460]);
source_activities = np.array([16.259,16.259,16.259,
                             11.646,11.646,
                              424.65]);#Bq/L
branching_ratio = np.array([0.11,0.86,0.94,
                           0.34,0.66,
                           0.11]);
activities_uncer = np.array([1.6259,1.6259,1.6259,
                            0.02329,0.02329,
                            0.8493])
'''The real source energies:
Lu-176: 88.34, 201.83, 306.78
La-138: 788.742,1435.795
K-40: 1460.83'''

In [ ]:
#roi = ROI.ROI(spectrum, background, source_energies, sub_type = 0)
#delta_e = 8
#window = np.array([[-2, -1], [-0.5, 0.5], [1, 2]])
#roi.set_sideband(delta_e,window)
#roi.find_peak_energies()
#target_peaks = roi.target_peaks
#roi_result,roi_uncer = roi.get_counts()

roi = ROI.ROI(spectrum, background, source_energies, 0)

roi.set_sideband(source_energies.tolist()[0], 8, [[-2, -1.1], [-0.4, 0.4], [1, 2]])
roi.set_sideband(source_energies.tolist()[1], 8, [[-2, -1], [-0.5, 0.5], [1, 2]])
roi.set_sideband(source_energies.tolist()[2], 8, [[-2, -1], [-0.5, 0.5], [1, 2]])
roi.set_sideband(source_energies.tolist()[3], 8, [[-2, -1], [-0.5, 0.5], [1, 2]])
roi.set_sideband(source_energies.tolist()[4], 8, [[-2, -1], [-0.6, 0.6], [1, 2]])
roi.set_sideband(source_energies.tolist()[5], 8, [[-2, -1], [-0.6, 0.6], [1, 2]])

roi.find_peak_energies()

In [ ]:
for i in range(len(source_energies.tolist())):
    roi.plot_peak_region(spectrum, source_energies, source_energies.tolist()[i])

In [ ]:
print(roi.roi_pars[f'75.0'])
fg_bins = roi.get_roi_windows(roi.roi_pars[f'75.0'],0)
bg_bins = roi.get_roi_windows(roi.roi_pars[f'75.0'],1)
print(fg_bins)
print(bg_bins)
print()
print(fg_bins[0][0][-1]-fg_bins[0][0][0])
print(fg_bins[1][0][-1]-fg_bins[1][0][0])
print(fg_bins[2][0][-1]-fg_bins[2][0][0])

np.sum(spectrum.counts_vals[fg_bins[0][0][0]:fg_bins[0][0][-1]])/(fg_bins[0][0][-1]-fg_bins[0][0][0])

In [ ]:
plt.plot(spectrum.bin_centers_kev[125:280],spectrum.counts_vals[125:280])
plt.plot(spectrum.bin_centers_kev[154:240],spectrum.counts_vals[154:240])
plt.show()

plt.plot(background.bin_centers_kev[150:310],background.counts_vals[150:310])
plt.plot(background.bin_centers_kev[178:277],background.counts_vals[178:277])
plt.show()

In [ ]:
roi.set_sideband(source_energies.tolist()[0], 8, [[-2, -1], [-0.4, 0.4], [1, 2]], 1)
roi.set_sideband(source_energies.tolist()[1], 8, [[-1.5, -0.5], [-0.5, 0.5], [1, 2]], 1)
roi.set_sideband(source_energies.tolist()[2], 8, [[-3, -2], [-0.5, 0.5], [1, 2]], 1)
roi.set_sideband(source_energies.tolist()[3], 8, [[-2, -1], [-0.5, 0.5], [1, 2]], 1)
roi.set_sideband(source_energies.tolist()[4], 8, [[-2, -1], [-0.6, 0.6], [1, 2]], 1)
roi.set_sideband(source_energies.tolist()[5], 8, [[-2, -1], [-0.6, 0.6], [1, 2]], 1)

for i in range(len(source_energies.tolist())):
    roi.plot_peak_region(background, source_energies, source_energies.tolist()[i], 1)

In [ ]:
target_peaks = roi.target_peaks
roi_result,roi_uncer = roi.get_counts()

#peakfit = PF.PF(spectrum,background,source_energies)
#pf_result = peakfit.get_counts()
print(target_peaks)
print('ROI result:',roi_result)
print('uncertainties:',roi_uncer)
#print('Lu_176 PF result:',pf_result)

In [ ]:
eff,eff_uncer = am.efficiency(roi_result,source_activities,branching_ratio,roi_uncer,activities_uncer,spectrum.livetime)
print(eff,eff_uncer)

In [ ]:
plot_eff = am.Efficiency(source_energies,eff,eff_uncer)

plot_eff.fitting()
plot_eff.plotter_pretty(0.04,save=True)

In [ ]:
plot_eff.save_fit()

In [ ]:
test_eff = am.Efficiency()
test_eff.set_parameters()

In [ ]:
test_eff.get_eff(1000)

In [ ]:
test_eff.get_eff_error(1000)

In [ ]:
test_eff.get_eff_error(2000)/plot_eff.get_eff(2000)

In [ ]:
test_eff.plotter_pretty()